In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

from sdm_eurec4a.visulization import set_custom_rcParams, label_from_attrs

set_custom_rcParams()

import matplotlib.pyplot as plt

In [ ]:
from pySD.sdmout_src import supersdata
from pySD.sdmout_src import pygbxsdat, pysetuptxt, supersdata

In [ ]:
data_dir = Path(
    "/home/m/m301096/CLEO/data/output_v4.1/condensation/combined/eulerian_dataset_combined.nc"
)
full_ds = xr.open_dataset(data_dir)

ds = full_ds.sel(time=slice(150, None))
ds

In [ ]:
data = (
    1e3
    * (ds["mass_represented"] / ds["gridbox_volume"])
    .mean(dim="time", keep_attrs=True, skipna=True)
    .compute()
)


fig, axs = plt.subplots(nrows=3, figsize=(8, 6))

for _ax in axs[:2]:
    _ax.plot(
        data.sum("radius_bins").T,
        ds["gridbox_coord3"].T,
    )

    _ax.set_ylabel("Height [m]")
    _ax.set_xlabel("Mass concentration [g/m³]")

axs[1].set_xscale("log")

axs[2].hist(
    1e2 * (data.sel(gridbox=1) / data.sel(gridbox=ds["max_gridbox"])).values.flatten(),
    bins=np.arange(0, 200, 10),
)

fig.tight_layout()

In [ ]:
data = 1e3 * (ds["mass_difference"] / ds["gridbox_volume"]).mean(dim="time", keep_attrs=True).compute()


fig, axs = plt.subplots(nrows=2, figsize=(8, 6))

for _ax in axs:
    _ax.plot(
        data.sum("radius_bins").T,
        ds["gridbox_coord3"].T,
    )

    _ax.set_ylabel("Height [m]")
    _ax.set_xlabel("Mass concentration [g/m³]")

axs[1].set_xscale("log")

fig.tight_layout()

/tmp/ipykernel_962289/2937134083.py:15: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  axs[1].set_xscale('log')


In [ ]:
single_data_dir = Path(
    "/home/m/m301096/CLEO/data/output_v4.1/condensation/cluster_110/processed/eulerian_dataset.nc"
)
single_ds = xr.open_dataset(single_data_dir)
ds = single_ds.sel(time=slice(150, None))
ds

In [ ]:
data = (
    1e3
    * (single_ds["mass_represented"] / single_ds["gridbox_volume"])
    .mean(dim="time", keep_attrs=True)
    .compute()
)


fig, axs = plt.subplots(ncols=2, width_ratios=[1, 0.25], figsize=(10, 5), sharey=True)

pcm = axs[0].pcolormesh(
    single_ds["radius_bins"], single_ds["gridbox_coord3"], data, shading="auto", cmap="Reds"
)
fig.colorbar(mappable=pcm, ax=axs[0], label="Mass concentration [g/m³]")

axs[0].set_xscale("log")
axs[0].set_ylabel("Height [m]")
axs[1].plot(
    data.sum("radius_bins"),
    single_ds["gridbox_coord3"],
)

axs[0].set_xlabel("Radius µm")

Text(0.5, 0, 'Radius µm')